<a href="https://colab.research.google.com/github/ChitranshBose/Research-Paper-Recommendation-System-IR-Project/blob/varun/extractive_summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import numpy as np
import pandas as pd
import json
import nltk
nltk.download('wordnet')
nltk.download('punkt')
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [25]:
pip install PyPDF2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [26]:
import re
import os
import shutil
from PyPDF2 import PdfReader
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from heapq import nlargest

In [27]:
shutil.rmtree('/content/download_data/', ignore_errors=True)
os.mkdir("/content/download_data/")

In [31]:
import PyPDF2
import xml.etree.ElementTree as ET
# Open the PDF file
pdf_file = open('/content/download_data/ppr.pdf', 'rb')
pdf_reader = PyPDF2.PdfReader(pdf_file)
text = ""
num_pages = len(pdf_reader.pages)
print(num_pages)
for i in range(num_pages):
  page = pdf_reader.pages[i]
  text += page.extract_text()

# Create an XML document
root = ET.Element('document')
for line in text.split('\n'):
  if line.strip() != '':
    ET.SubElement(root, 'line').text = line

xml_file = open('example.xml', 'wb')
s=ET.tostring(root)
xml_file.write(ET.tostring(root))
xml_file.close()

6


In [ ]:
s=str(s)
#s=s.lower()
l=s.split("</line><line>")
l

In [33]:
lst = []
# z=[]

# all_small=['discussion', 'literature','developmental model', 'future scope', 'model','result', 'methodology', 'appendices', 'results', 'abstract', 'introduction', 'conclusion', 'related works','related', 'conclusions', 'experiments', 'background', 'references', 'developmental model', 'acknowledgments', 'experimental setup']
# all_caps=['DISCUSSION', 'LITERATURE', 'FUTURE SCOPE','MODEL' ,'RESULT', 'METHODOLOGY', 'APPENDICES', 'RESULTS', 'ABSTRACT', 'INTRODUCTION', 'CONCLUSION', 'RELATED','RELATED WORKS' , 'CONCLUSIONS', 'EXPERIMENTS', 'BACKGROUND', 'REFERENCES', 'DEVELOPMENTAL MODEL', 'ACKNOWLEDGMENTS', 'EXPERIMENTAL SETUP']
# first_caps=['Discussion', 'Literature', 'Future Scope', 'Developmental Model','Model','Result', 'Methodology', 'Appendices', 'Results', 'Abstract', 'Introduction', 'Conclusion', 'Related Works','Related', 'Conclusions', 'Experiments', 'Background', 'References', 'Developmental Model', 'Acknowledgments', 'Experimental Setup']

# for i in all_small:
#   z.append(i)

# for i in all_caps:
#   z.append(i)

# for i in first_caps:
#   z.append(i)
# print(len(z))

z=['INTRODUCTION','I.','II.','V.','UTURE',' NTRODUCTION','DEVELOPMENTAL','EXPERIMENTS','RELATED','WORKS','CONCLUSION','REFERENCES']

for x in l:
 line = x.split()
 if len(line) <=7:
  for i in line:
   if i in z:
    lst.append(line)
    continue
  if len(line)==1 and line[0][0].isupper():
    print(line)
lst

['Robots.']


[['I.', 'I', 'NTRODUCTION'],
 ['II.', 'R', 'ELATED', 'WORKS', 'AND', 'BACKGROUND'],
 ['II.', 'R', 'ELATED', 'WORKS', 'AND', 'BACKGROUND'],
 ['V.', 'C', 'ONCLUSION'],
 ['VI.', 'F', 'UTURE', 'WORKS'],
 ['VI.', 'F', 'UTURE', 'WORKS']]

In [34]:
intro=""
conc=""
for x in range(len(l)):
  line=l[x].split()
  if(len(line)<=6):
    for i in line:
      if 'NTRODUCTION' in i:
        print("hi",x)
        intro_index=x
      if 'ELATED' in i:
        print("bye",x)
        seco_index=x
      if 'V.' in i:
        print("hi",x)
        conc_index=x
      if 'VI.' in i:
        print("bye",x)
        seco_conc_index=x

conc_index,seco_conc_index

hi 34
bye 104
hi 244
hi 338
bye 371


(338, 371)

In [35]:
for i in range(intro_index+1,seco_index):
  intro=intro+" "+l[i]

print(len(intro))
print(intro)

for i in range(conc_index+1,seco_conc_index):
  conc=conc+" "+l[i]

print(len(conc))
print(conc)

4266
 Autonomous vehicles are an emerging technology that is gaining popularity [1]. They are able to recognise the scene, plan the path, and manage the motion independently when interacting with drivers. Recent technological advancements have made it possible for fully autonomous vehicles to become a reality. Fig. 1. Autonomous Vehicle revolution Significant advancements have been made in self-driving automobile technology over the past ten years. Even thoughthe Self driving technologies bring more benefits, there are concerns about it&#8217;s reliability and road users safety which need to be addressed. Automobiles travel 10 trillion miles worldwide each year, creating millions of scenarios where autonomous vehicles could malfunction. However, there are several obstacles at every stage of system operation. As with most scientific and technological successes, re- search has contributed creative and unique ideas, proof-of- concept systems, and a steady stream of bright engineers and en

In [36]:
def summarize(text, per):
    nlp = spacy.load('en_core_web_sm')
    doc= nlp(text)
    tokens=[token.text for token in doc]
    wf={}
    for word in doc:
        if word.text.lower() not in list(STOP_WORDS):
            if word.text.lower() not in punctuation:
                if word.text not in wf.keys():
                    wf[word.text] = 1
                else:
                    wf[word.text] += 1
    f=max(wf.values())
    for word in wf.keys():
        wf[word]=wf[word]/f
    s= [sent for sent in doc.sents]
    score = {}
    for sent in s:
        for word in sent:
            if word.text.lower() in wf.keys():
                if sent not in score.keys():                            
                    score[sent]=wf[word.text.lower()]
                else:
                    score[sent]+=wf[word.text.lower()]
    select_length=int(len(s)*per)
    summary=nlargest(select_length, score,key=score.get)
    final_summary=[word.text for word in summary]
    summary=''.join(final_summary)
    return summary



In [37]:
summarize(intro,0.5)

'When a vehicle is in autonomous mode, for example, a vehicle control system can handle some or all of the driving aspects of vehicle operation.During autonomous vehicle driving, the vehicle may detect a circumstance in which confidence in operation falls below a predetermined threshold.As with most scientific and technological successes, re- search has contributed creative and unique ideas, proof-of- concept systems, and a steady stream of bright engineers and entrepreneurs to creating the technology that makes au- tonomous driving and highly automated vehicles a reality.planning a driving path, sensing aspects of the vehicle, sensing the vehicle&#8217;s environment, and managing drive components like steering throttle, and brake in such cases.The characteristics of these vehicles have made intelligent systems orientation and real- world experiments crucial to the advancement of the field of autonomous driving, not just in academic research groups but also, more recently, in industry.

In [38]:
summarize(conc,0.5)

'When the vehicle is entirely driverless, i.e., there is no driver present, the circumstance necessitates the use of a human operator to remotely drive the vehicle.The robot is guided to a safe area by the remote operator via camera feed and teleoperation using keyboard, afterwards the A V returns to its autonomous mode.Using the ROS Master URI configuration, we have established a remote connection between the turtlebot3 system and the remote-control system.The purpose of this demonstrator was to simulate a real- world situation and test the method of remotely operating autonomous cars using an autonomous robot.driving cars to be integrated onto the roads before they are completely autonomous in all conditions, without the need for a safety driver in the vehicle.The remote operator reports the issue and provides feedback for switching to manual control.This is not an issue when there is a driver in the car; the driver can take control of the vehicle.'

In [39]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 48.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 62.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 23.9 MB/s eta 0:00:00


In [40]:
from transformers import pipeline 
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
print(summarizer(intro, max_length=200, min_length=100, do_sample=False))

[{'summary_text': 'Autonomous vehicles are an emerging technology that is gaining popularity. They are able to recognise the scene, plan the path, and manage the motion independently when interacting with drivers. Around 1.5 million people lose their lives in traffic accidents every year. Around 11% of all fatalities worldwide are caused by vehicle accidents. Autonomous cars have the potential to minimize or eliminate the requirement for human interaction in different aspects of vehicle operation. When an autonomous vehicle&#8217;s confidence in its ability to operate is limited, it will request assistance from a remote operator.'}]


In [41]:
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
print(summarizer(conc, max_length=200, min_length=100, do_sample=False))

[{'summary_text': 'Self-driving vehicles will inevitably encounter complex scenarios in which they may be unable to make judgments on their own. This necessitates a driver taking control of the vehicle until it can drive itself again. The purpose of this demonstrator was to simulate a real- world situation and test the method of remotely operating autonomous cars using an autonomous robot. Using the ROS Master URI configuration, we have established a remote connection between the turtlebot3 system and the remote-control system. The robot is guided to a safe area by the remote operator via camera feed and teleoperation using keyboard.'}]


In [ ]:
from transformers import BigBirdPegasusForConditionalGeneration, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google/bigbird-pegasus-large-arxiv")
model1 = BigBirdPegasusForConditionalGeneration.from_pretrained("google/bigbird-pegasus-large-arxiv")
inputs = tokenizer(conc, return_tensors='pt')
prediction1 = model1.generate(**inputs)
prediction1 = tokenizer.batch_decode(prediction1)
print(prediction1)

In [ ]:
model2 = BigBirdPegasusForConditionalGeneration.from_pretrained("google/bigbird-pegasus-large-arxiv", attention_type="original_full")
prediction2 = model2.generate(**inputs)
prediction2 = tokenizer.batch_decode(prediction2)
print(prediction2)

In [ ]:
model3 = BigBirdPegasusForConditionalGeneration.from_pretrained("google/bigbird-pegasus-large-arxiv", block_size=16, num_random_blocks=2)
prediction3 = model3.generate(**inputs)
prediction3 = tokenizer.batch_decode(prediction3)
print(prediction3)